In [89]:
from imdbpie import Imdb
import os
import charts_function_list

In [90]:
_,datafolder,outputs=charts_function_list.folder_setup()

In [6]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

In [123]:
def title_search(title):
    return pd.DataFrame(imdb.search_for_title(title))

In [128]:
combined_titles = pd.concat([title_search(title) for title in titles_to_search])

#clean data
combined_titles.dropna(subset=['year'],inplace=True)
combined_titles['year']=combined_titles['year'].astype('int')
combined_titles[(combined_titles['year']<=2017)&(combined_titles['year']>1970)].reset_index(drop=True)

#get ratings
combined_titles['rating']=[imdb.get_title_by_id(title_).rating for title_ in combined_titles['imdb_id']]
combined_titles['votes']=

In [172]:
pd.DataFrame({'Rating':[imdb.get_title_by_id(title_).rating for title_ in combined_titles['imdb_id'],
                        'Votes':[imdb.get_title_by_id(title_).votes for title_ in combined_titles['imdb_id']

,A,B
0,2,2
1,3,3
2,4,4


In [166]:
a

'A'

In [154]:
combined_titles.dropna(subset=['year'],inplace=True)
combined_titles['year']=combined_titles['year'].astype('int')
combined_titles[(combined_titles['year']<=2017)&(combined_titles['year']>1970)]

,imdb_id,title,year
0,tt0087928,Police Academy,1984
1,tt0089822,Police Academy 2: Their First Assignment,1985
2,tt0091777,Police Academy 3: Back in Training,1986
4,tt0093756,Police Academy 4: Citizens on Patrol,1987
5,tt0095882,Police Academy 5: Assignment: Miami Beach,1988
6,tt0098105,Police Academy 6: City Under Siege,1989
7,tt0110857,Police Academy: Mission to Moscow,1994
8,tt0094526,Police Academy: The Series,1988
9,tt0126168,Police Academy: The Series,1997
10,tt4325152,Police Academy Alaturka,2015


In [151]:
combined_titles

,imdb_id,title,year
0,tt0087928,Police Academy,1984
1,tt0089822,Police Academy 2: Their First Assignment,1985
2,tt0091777,Police Academy 3: Back in Training,1986
4,tt0093756,Police Academy 4: Citizens on Patrol,1987
5,tt0095882,Police Academy 5: Assignment: Miami Beach,1988
6,tt0098105,Police Academy 6: City Under Siege,1989
7,tt0110857,Police Academy: Mission to Moscow,1994
8,tt0094526,Police Academy: The Series,1988
9,tt0126168,Police Academy: The Series,1997
10,tt4325152,Police Academy Alaturka,2015
